In [5]:
from keras import models
from keras import layers
from keras.utils import np_utils, generic_utils
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG19
from sklearn.model_selection import KFold
import pickle

In [2]:
### Get data from the input file ###

infile = open('S:\\Usyd\\Project\\Data\\train_data','rb')
Images = pickle.load(infile)
Labels = pickle.load(infile)
Patientid = pickle.load(infile)
infile.close()

In [7]:
### Data ###

# Convert labels into binary #
binaryLabels = np_utils.to_categorical(Labels)
print(len(Labels))
print(len(Images))

# Training data #
xtrain = Images
ytrain = binaryLabels
print(len(xtrain))
print(len(ytrain))

# Define Data Augmentation function #
def data_aug(X_train,X_test,y_train,y_test,train_batch_size,test_batch_size):
    train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    test_datagen = ImageDataGenerator(rescale=1.0/255)
    train_batch = train_datagen.flow(X_train,y_train,batch_size=train_batch_size)
    test_batch = test_datagen.flow(X_test,y_test,batch_size=test_batch_size)
    return (train_batch,test_batch)

196
196
196
196


In [8]:
base_model = VGG19(weights=None, include_top=False, input_shape=(224,224,3))

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
acc_per_fold = []       # Define per-fold score containers
loss_per_fold = []      # Define per-fold score containers

fold_no = 1
for train, val in kfold.split(xtrain, ytrain):
    X_train = xtrain[train]
    X_val = xtrain[val]
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    y_train = ytrain[train]
    y_val = ytrain[val]
    BS = 15
    train_batch, val_batch = data_aug(X_train,X_val,y_train,y_val, BS, BS)
    

    
    # Model architecture #
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))
    model.summary()
    
    # Extract the weights before training #
    weights_untrained = model.layers[0].get_weights()[0]
    
    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    BS = 15
    Epochs_no = 30
    

    # Train the model 
    model.fit_generator(train_batch, validation_data=val_batch, epochs=Epochs_no, validation_steps= X_val.shape[0] // BS, 
                        steps_per_epoch= X_train.shape[0] // BS, verbose=2)
    
    # Extract the weights after training
    weights_trained = model.layers[0].get_weights()[0]
    print(weights_trained.shape)
    
    # Save model #
    model_name = 'VGG19_Fold'+str(fold_no)+'.h5'
    model.save(model_name)
    
    # Generate generalization metrics
    scores = model.evaluate(X_val, y_val)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')